### Random Shuffling Function

In [ ]:
import pandas as pd

original_df = pd.read_csv("processed_data/sequences/reuters_eventlogs.csv")
original_df


In [ ]:
lists = original_df.groupby('I').apply(lambda group: [tuple(row)[1:4] for row in group.itertuples(index=False)]).to_list()
for l in lists:
    l.sort(key=lambda x: x[0])

In [ ]:
from random import choice

def random_from_lists(lists, random_length = True):
    length = len(lists)
    r = []
    j = 0
    selected_length = len(lists[choice(range(length))])
    while True:
        if (random_length and j >= selected_length):
            break
        i = choice(range(length))
        if j < len(lists[i]) :
            if j == 0:
                r.append(lists[i][j])
            else:
                r.append((r[j - 1][0] + (lists[i][j][0] - lists[i][j-1][0]), lists[i][j][1], lists[i][j][2]))  
            j = j + 1   
        else:
            if not random_length:
                break
    return r         

In [ ]:
pd.Series([len(random_from_lists(lists)) for _ in range(len(lists))]).describe()

In [ ]:
pd.Series([len(l) for l in lists ]).describe()

In [ ]:
event_log = random_from_lists(lists)
event_log

### recompute_values_according_to_labels

In [ ]:
def recompute_values_according_to_labels(event_log, type="Remaining"):
    result = []

    for i, data, _ in event_log:
        remaining_count = 0
        total_occurrences = 0

        for _ , other_data, _ in event_log[i + 1:]:
            if other_data == data:
                remaining_count += 1
                total_occurrences = remaining_count + 1

        if type == "Remaining":
            result.append((i, data, remaining_count))
        elif type == "Percentage_Alpha":
            total_remaining = len(event_log[i + 1:])
            percentage_count = 0 if total_remaining == 0 else remaining_count / total_remaining
            result.append((i, data, percentage_count))
        elif type == "Percentage_Beta":
            count_value = 0 if total_occurrences == 0 else remaining_count / total_occurrences
            result.append((i, data, count_value))
        else:
            raise ValueError("Invalid type. Use 'Remaining', 'Percentage_Alpha', or 'Percentage_Beta'.")

    return result


In [ ]:
#Example Usage

# Using type="Remaining"
result_remaining = recompute_values_according_to_labels(event_log, type="Remaining")
print("Result (Remaining):", result_remaining)

# Using type="Percentage_Alpha"
result_percentage_alpha = recompute_values_according_to_labels(event_log, type="Percentage_Alpha")
print("Result (Percentage_Alpha):", result_percentage_alpha)

# Using type="Percentage_Beta"
result_percentage_beta = recompute_values_according_to_labels(event_log, type="Percentage_Beta")
print("Result (Percentage_Beta):", result_percentage_beta)